In [22]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk import sent_tokenize, word_tokenize
import operator

from nervaluate import Evaluator
from nltk.tokenize import sent_tokenize

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/matijagercer/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/matijagercer/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /Users/matijagercer/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/matijagercer/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [23]:
def read_text(path):
    with open(path, encoding="utf-8") as f:
        text = f.read()
        text = text.replace('\r', ' ').replace('\n', ' ')\
            .replace("’", "'").replace("\"", "").replace("”", "").replace("“", "")
    return text

def nltk_NER(book, eval = False):
    """
    nlkt_NER vrne seznam, v katerem so shranjene prepoznane identitete glede na posamezni stavek
    :param book: str
    :return: entity_dict (seznam slovarjev kot npr. [name, tag, start_pos, stop_pos, line_num. token_num])
    """
    
    # EVAL ---
    if eval == True:
        sentences, sentences_tag = conll_read()
        #print(len(sentences), "--------")
    else:
        sentences = sent_tokenize(book)
    sentences_NER = []
    sentences_NER_results = []
    # EVAL ---
    
    entity_dict = []

    for line_num, line in enumerate(sentences):
        
        # EVAL ---
        words_list = word_tokenize(line)
        tags_list = ["O"] * len(words_list)
        sentences_NER.append(words_list)
        # EVAL ---

        words = nltk.word_tokenize(line)
        pos_tag = nltk.pos_tag(words)
        ne_chunk = nltk.ne_chunk(pos_tag, binary=False)

        token_num = 0
        for chunk in ne_chunk:
            if hasattr(chunk,'label'):
                start_pos = token_num
                stop_pos = token_num + len(chunk)
                tag = chunk.label()
                name = ' '.join(c[0] for c in chunk)
                token_num += len(chunk)

                info_dict = {}
                info_dict["name"] = name
                info_dict["tag"] = tag
                info_dict["start_pos"] = start_pos
                info_dict["stop_pos"] = stop_pos
                info_dict["line_num"] = line_num
                entity_dict.append(info_dict)
                
                #print(name, tag)
                # EVAL: ---
                #names = word.text.replace("'s", "")
                names = name.split(" ")
                ff_flag = False
                if tag == "PERSON":
                    for name in names:
                        try:
                            if ff_flag == False:
                                idx = words_list.index(name)
                                tags_list[idx] = "B-PER"
                                ff_flag = True
                            else:
                                idx = words_list.index(name)
                                tags_list[idx] = "I-PER"
                        except:
                            print("Not found: ", name)
                # EVAL ---
            
                
            else:
                token_num += 1
        sentences_NER_results.append(tags_list) 


    return entity_dict, sentences_NER, sentences_NER_results


def get_names_from_NER(entity_dict):
    """
    get_names_from_NER sprejme entity_dict in vrne urejen seznam terk ("ime", št_zaznano)
    :param entity_dict: dict (seznam dictov)
    :return: unique_names: list
    """
    unique_names = {}

    for entity in entity_dict:
        if entity["tag"] == "PERSON":
            if entity["name"] not in unique_names:
                unique_names[entity["name"]] = 1
            else:
                unique_names[entity["name"]] += 1
    unique_names = sorted(unique_names.items(), key=operator.itemgetter(1),reverse=True)

    return unique_names


def conll_read():
    with open("conll2003/valid.txt") as f:
        lines = f.readlines()
    sentences = []
    sentences_tag = []
    tmp = ""
    tmp_tag = ""
    tags = []
    for i in range(len(lines)):

        line = lines[i]
        line = line.split(" ")
        text_word = line[0]

        if text_word[0] != "-":
            if text_word == "\n":
                # print(tmp)
                # print(tmp_tag)
                # print("-------")
                sentences.append(tmp)
                tmp = ""
                #sentences_tag.append(tmp_tag)
                #tmp_tag = ""
                sentences_tag.append(tags)
                tags = []

            elif text_word == "(" or text_word == ")" or text_word == '"' or text_word == ':':
                pass
            else:
                if text_word[0] != "." and text_word[0] != "," and text_word[0] != ":" and text_word[0] != ";"\
                        and text_word[0] != "!" and text_word[0] != "?" and text_word[0] != ")" and text_word[0] != "'":   #or text_word[0] != "," or text_word[0] != "(" or text_word[0] != ")":
                    tmp = tmp + " " + text_word
                    line[3] = line[3].replace("\n", "")

                    if line[3] == "B-PER" or line[3] == "I-PER":
                        #line[3] = "B-PER"
                        #tmp_tag = tmp_tag + " " + line[3]
                        tags.append(line[3])
                    else:
                        line[3] = "O"
                        #tmp_tag = tmp_tag + " " + line[3]
                        tags.append(line[3])
                else:
                    tmp = tmp +  text_word
                    line[3] = "O"
                    tags.append(line[3])
    return sentences, sentences_tag

In [24]:
book = read_text('../../data/books/ASongOfIceAndFire/AGOT/chapters/1_Bran_1.txt')
entity_dict, sentences_NER, sentences_NER_results = nltk_NER(book, False)
unique_names = get_names_from_NER(entity_dict)

for (name, num) in unique_names:
    if True: #num > 1:
        print(name, num)


Bran 42
Jon 29
Robb 27
Father 8
Jory 7
Greyjoy 5
Hullen 4
Snow 3
Lord 2
Robert 2
Jon Snow 2
Theon Greyjoy 2
Mance Rayder 1
Old Nan 1
Eddard Stark 1
Jory Cassel 1
Blood 1
Stark 1
Old 1
Nan 1
Watch 1
Harwin 1
Ser 1
Rodrik 1
Rickon 1
Desmond 1


In [19]:
# Evalvacija na datasetu Conll2003 (osredotočena na imena)

def eval_wrapper():
    entity_dict, sentences_NER, sentences_NER_results = nltk_NER("", eval= True)
    unique_names = get_names_from_NER(entity_dict)

    # Kontrola podatkov
    sentences_org, sentences_org_tag = conll_read()
    # for x in range(len(sentences_NER)):
    #     print("-----------", x)
    #     #print(len(sentences_org_tag[x]))
    #     print(len(sentences_NER[x]))
    #     print(len(sentences_NER_results[x]))

    #     #print(sentences_org[x])    # 22 26
    #     print(sentences_NER[x])
    #     print(sentences_org_tag[x])
    #     print(sentences_NER_results[x])
    
    true = sentences_org_tag
    pred = sentences_NER_results
    evaluator = Evaluator(true, pred, tags=['PER'], loader="list")

    results, results_by_tag = evaluator.evaluate()
    print(results)

In [26]:
## Evalvacija za I. poglavje GoT

from chapter__1annotated import chapter1_lst

# Kontrola:
# print(len(sentences_NER_results))
# print(len(chapter1_lst))

# for x in range(len(sentences_NER)):
#     print("-----------", x)
#     print(len(sentences_NER[x]))
#     print(len(sentences_NER_results[x]))
#     print(len(chapter1_lst[x]))
    

#     print(sentences_NER[x])
#     print(sentences_NER_results[x])
#     print(chapter1_lst[x])

true = chapter1_lst
pred = sentences_NER_results
evaluator = Evaluator(true, pred, tags=['PER'], loader="list")

results, results_by_tag = evaluator.evaluate()
print(results)

{'ent_type': {'correct': 137, 'incorrect': 0, 'partial': 0, 'missed': 27, 'spurious': 7, 'possible': 164, 'actual': 144, 'precision': 0.9513888888888888, 'recall': 0.8353658536585366, 'f1': 0.8896103896103895}, 'partial': {'correct': 128, 'incorrect': 0, 'partial': 9, 'missed': 27, 'spurious': 7, 'possible': 164, 'actual': 144, 'precision': 0.9201388888888888, 'recall': 0.8079268292682927, 'f1': 0.8603896103896104}, 'strict': {'correct': 128, 'incorrect': 9, 'partial': 0, 'missed': 27, 'spurious': 7, 'possible': 164, 'actual': 144, 'precision': 0.8888888888888888, 'recall': 0.7804878048780488, 'f1': 0.8311688311688312}, 'exact': {'correct': 128, 'incorrect': 9, 'partial': 0, 'missed': 27, 'spurious': 7, 'possible': 164, 'actual': 144, 'precision': 0.8888888888888888, 'recall': 0.7804878048780488, 'f1': 0.8311688311688312}}
